# Data Analysis

**<code> by ZaraPhCode</code>**

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, date
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler
import seaborn as sns # for plotting graphs
import numpy as np
np.set_printoptions(precision=2)

## reading the file

First, we open the <code>.xlsx</code> file using the function <code>read_excel</code> from <code>Pandas</code>. It copy the data of the excel file sheets into a dictionary. In roder to read each sheet, we assign a DataFrame to each dictionary key.

In [91]:
xlsx_path = 'KPMG_VI_New_raw_data_update_final.xlsx'
sheet_names = ['Transactions','NewCustomerList','CustomerDemographic','CustomerAddress']
df = []

df_read = pd.read_excel(xlsx_path, sheet_name=None)
df.append(df_read.get('Transactions'))
df.append(df_read.get('NewCustomerList'))
df.append(df_read.get('CustomerDemographic'))
df.append(df_read.get('CustomerAddress'))

See first elements of the sheet <code>Transactions</code>

In [3]:
df[0].head()

,Note: The data and information in this document is reflective of a hypothetical situation and client. This document is to be used for KPMG Virtual Internship purposes only.,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
1,1,2,2950,2017-02-25 00:00:00,False,Approved,Solex,Standard,medium,medium,71.49,53.62,41245
2,2,3,3120,2017-05-21 00:00:00,True,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,41701
3,3,37,402,2017-10-16 00:00:00,False,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,36361
4,4,88,3135,2017-08-31 00:00:00,False,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.1,36145


As the titles are not correctly assigned as the column titles, we extract the columns correct name and put them into a list:

In [92]:
titles = []
for i in range(len(sheet_names)):
    titles.append(list(df[i].iloc[0,:]))

titles[0]

['transaction_id',
 'product_id',
 'customer_id',
 'transaction_date',
 'online_order',
 'order_status',
 'brand',
 'product_line',
 'product_class',
 'product_size',
 'list_price',
 'standard_cost',
 'product_first_sold_date']

We then rename the columns:

In [93]:
for i in range(len(sheet_names)):
    df[i].columns =titles[i]
    df[i] = df[i].loc[1:,:]
    df[i].reset_index(inplace=True)
    df[i] = df[i].iloc[:,1:]  

initial cleaning the dataframes:

In [6]:
df[2].head()


,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,default,owns_car,tenure
0,1,Laraine,Medendorp,F,93,1953-10-12 00:00:00,Executive Secretary,Health,Mass Customer,N,"""'",Yes,11
1,2,Eli,Bockman,Male,81,1980-12-16 00:00:00,Administrative Officer,Financial Services,Mass Customer,N,<script>alert('hi')</script>,Yes,16
2,3,Arlin,Dearle,Male,61,1954-01-20 00:00:00,Recruiting Manager,Property,Mass Customer,N,2018-02-01 00:00:00,Yes,15
3,4,Talbot,NaN,Male,33,1961-10-03 00:00:00,NaN,IT,Mass Customer,N,() { _; } >_[$($())] { touch /tmp/blns.shellsh...,No,7
4,5,Sheila-kathryn,Calton,Female,56,1977-05-13 00:00:00,Senior Editor,NaN,Affluent Customer,N,NIL,Yes,8


counting the items in each column:

In [7]:
df[2]['gender'].value_counts()

gender
Female    2037
Male      1872
U           88
F            1
Femal        1
M            1
Name: count, dtype: int64

all of them are related to 2000 and later

Lets see the dimension of each data sheet:

In [94]:
for i in range(len(sheet_names)):
    print(df[i].shape)

(20000, 13)
(1000, 23)
(4000, 13)
(3999, 6)


Lets delete duplicate rows:

In [95]:
for i in range(len(sheet_names)):
    df[i] = df[i].drop_duplicates()
    print(df[i].shape)


(20000, 13)
(1000, 23)
(4000, 13)
(3999, 6)


## NOTE:
There is no duplicated row in each sheet

Lets see how many nul data we have, for example in the <code>Transaction</code> sheet:

In [11]:
df[0].head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
0,1,2,2950,2017-02-25 00:00:00,False,Approved,Solex,Standard,medium,medium,71.49,53.62,41245
1,2,3,3120,2017-05-21 00:00:00,True,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,41701
2,3,37,402,2017-10-16 00:00:00,False,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,36361
3,4,88,3135,2017-08-31 00:00:00,False,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.1,36145
4,5,78,787,2017-10-01 00:00:00,True,Approved,Giant Bicycles,Standard,medium,large,1765.3,709.48,42226


In [10]:
df[0].shape[0]

20000

In [11]:
df[0].count(axis='columns')[0]

13

In [12]:
print(df[0].shape)
print(df[0].isnull().sum())

(20000, 13)
transaction_id               0
product_id                   0
customer_id                  0
transaction_date             0
online_order               360
order_status                 0
brand                      197
product_line               197
product_class              197
product_size               197
list_price                   0
standard_cost              197
product_first_sold_date    197
dtype: int64


In [96]:
summary_stats = []

for i in range(len(sheet_names)):
    df[i] = df[i].dropna()
    df[i] = df[i].loc[:, ~df[i].columns.isnull()]
    summary_stats.append(df[i].describe())
    print(summary_stats[i])
    print('---------------------------')

        transaction_id  product_id  customer_id     transaction_date  \
count            19445       19445        19445                19445   
unique           19445         101         3492                  364   
top                  1           0         1068  2017-08-18 00:00:00   
freq                 1        1157           14                   81   

       online_order order_status  brand product_line product_class  \
count         19445        19445  19445        19445         19445   
unique            2            2      6            4             3   
top            True     Approved  Solex     Standard        medium   
freq           9739        19273   4169        13920         13587   

       product_size  list_price  standard_cost  product_first_sold_date  
count         19445    19445.00       19445.00                    19445  
unique            3      100.00         103.00                      100  
top          medium     2091.47         388.92                    

In [9]:
print(df[0].isnull().sum())

transaction_id             0
product_id                 0
customer_id                0
transaction_date           0
online_order               0
order_status               0
brand                      0
product_line               0
product_class              0
product_size               0
list_price                 0
standard_cost              0
product_first_sold_date    0
dtype: int64


Remove outliers in DOB:

In [97]:
df[2]['DOB'] = pd.to_datetime(df[2]['DOB'], format='%Y-%m-%d')
df[1]['DOB'] = pd.to_datetime(df[1]['DOB'], format='%Y-%m-%d')
  
# Filter data between two dates
df[2]['DOB'] = df[2]['DOB'].loc[df[2]['DOB'] >= '1920-01-01']
df[1]['DOB'] = df[1]['DOB'].loc[df[1]['DOB'] >= '1920-01-01']
df[1].shape

(715, 18)

In [55]:
df[2]['wealth_segment'].value_counts()

wealth_segment
Mass Customer        1300
High Net Worth        674
Affluent Customer     656
Name: count, dtype: int64

In [134]:
df[0] = df[0].loc[df[0]["order_status"] == 'Approved' ]

df[2]['job_industry_category'] = df[2]['job_industry_category'].replace(['Argiculture'], 'Agriculture')
df[1]['job_industry_category'] = df[1]['job_industry_category'].replace(['Argiculture'], 'Agriculture')

# Replace values in the 'gender' column
df[1]['gender'] = df[1]['gender'].replace(['F', 'femal','female','Femal'], 'Female')
df[2]['gender'] = df[2]['gender'].replace(['F', 'femal','female','Femal'], 'Female')

df[3]['state'] = df[3]['state'].replace(['New South Wales'], 'NSW')
df[3]['state'] = df[3]['state'].replace(['Victoria'], 'VIC')

# Add 'gender' column to df1 based on customer_id mapping
gender_mapping = df[2].set_index('customer_id')['gender'].to_dict()
df[0]['gender'] = df[0]['customer_id'].map(gender_mapping)
# df[0]['gender'] = df[0]['gender'].replace(['F','femal','Femal','female'],'Female')

# Add 'wealth_segment' column to df1 based on customer_id mapping
wealth_segment_mapping = df[2].set_index('customer_id')['wealth_segment'].to_dict()
df[0]['wealth_segment'] = df[0]['customer_id'].map(wealth_segment_mapping)

# Add 'age' column to df1 based on customer_id mapping
df[2]['age'] = pd.to_datetime(df[2]['DOB'], format='%m %d %y')
current_year = datetime.now().year
df[2]['age'] = current_year - df[2]['age'].dt.year
age_mapping = df[2].set_index('customer_id')['age'].to_dict()
df[0]['age'] = df[0]['customer_id'].map(age_mapping)

df[1]['age'] = pd.to_datetime(df[1]['DOB'], format='%Y-%m-%d')
df[1]['age'] = current_year - df[1]['age'].dt.year


# Add 'transaction date' column to df1 based on customer_id mapping
df[0]['transaction_date_datetime'] = pd.to_datetime(df[0]['transaction_date']).dt.date

# Add 'job_category' column to df1 based on customer_id mapping
job_category_mapping = df[2].set_index('customer_id')['job_industry_category'].to_dict()
df[0]['job_industry_category'] = df[0]['customer_id'].map(job_category_mapping)

# Add 'ownes_car' column to df1 based on customer_id mapping
car_mapping = df[2].set_index('customer_id')['owns_car'].to_dict()
df[0]['owns_car'] = df[0]['customer_id'].map(car_mapping)

# Add 'part3year' column to df1 based on customer_id mapping
part3year_mapping = df[2].set_index('customer_id')['past_3_years_bike_related_purchases'].to_dict()
df[0]['past_3_years_bike_related_purchases'] = df[0]['customer_id'].map(part3year_mapping)

# Add 'country' column to df1 based on customer_id mapping
country_mapping = df[3].set_index('customer_id')['country'].to_dict()
df[0]['country'] = df[0]['customer_id'].map(country_mapping)

# Add 'state' column to df1 based on customer_id mapping
state_mapping = df[3].set_index('customer_id')['state'].to_dict()
df[0]['state'] = df[0]['customer_id'].map(state_mapping)

# Add 'postcode' column to df1 based on customer_id mapping
postcode_mapping = df[3].set_index('customer_id')['postcode'].to_dict()
df[0]['postcode'] = df[0]['customer_id'].map(postcode_mapping)


df[0].head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,...,gender,wealth_segment,age,transaction_date_datetime,job_industry_category,owns_car,past_3_years_bike_related_purchases,country,state,postcode
0,1,2,2950,2017-02-25,False,Approved,Solex,Standard,medium,medium,...,Male,Mass Customer,68.0,2017-02-25,Financial Services,Yes,19.0,Australia,VIC,3064.0
1,2,3,3120,2017-05-21,True,Approved,Trek Bicycles,Standard,medium,large,...,Female,Mass Customer,44.0,2017-05-21,Health,Yes,89.0,Australia,NSW,2196.0
2,3,37,402,2017-10-16,False,Approved,OHM Cycles,Standard,low,medium,...,Male,Affluent Customer,46.0,2017-10-16,Retail,No,9.0,Australia,NSW,2835.0
3,4,88,3135,2017-08-31,False,Approved,Norco Bicycles,Standard,medium,medium,...,Male,Mass Customer,61.0,2017-08-31,Financial Services,No,83.0,Australia,NSW,2096.0
4,5,78,787,2017-10-01,True,Approved,Giant Bicycles,Standard,medium,large,...,NaN,NaN,NaN,2017-10-01,NaN,NaN,NaN,Australia,NSW,2292.0


In [57]:
df[0]['customer_id'].shape

(19273,)

In [58]:
print(df[0].isnull().sum())

transaction_id                            0
product_id                                0
customer_id                               0
transaction_date                          0
online_order                              0
order_status                              0
brand                                     0
product_line                              0
product_class                             0
product_size                              0
list_price                                0
standard_cost                             0
product_first_sold_date                   0
gender                                 6418
wealth_segment                         6418
age                                    6418
transaction_date_datetime                 0
job_industry_category                  6418
owns_car                               6418
past_3_years_bike_related_purchases    6418
country                                  28
state                                    28
postcode                        

In [59]:
df[0]['transaction_date'].max()

datetime.datetime(2017, 12, 30, 0, 0)

calculate the maximum transaction date for each customer using groupby() and max():

In [99]:
df_data = pd.DataFrame(columns = ['customer_id', 'age', 'gender', 'job_industry_category', 'owns_car', 'past_3_years_bike_related_purchases', 'state','postcode', 'wealth_segment','Recency','Profit','Frequency'])
df_data['customer_id'] = df[0]['customer_id']

df_data['gender'] = df[0]['gender']
df_data['Profit'] = df[0]['list_price'] - df[0]['standard_cost']
df_data['age'] = df[0]['age']
df_data['job_industry_category'] = df[0]['job_industry_category']
df_data['owns_car'] = df[0]['owns_car']
df_data['past_3_years_bike_related_purchases'] = df[0]['past_3_years_bike_related_purchases']
df_data['state'] = df[0]['state']
df_data['postcode'] = df[0]['postcode']
df_data['wealth_segment'] = df[0]['wealth_segment']
df_data['Frequency'] = df[0].groupby('customer_id')['customer_id'].transform('count').astype(float)

specific_date = pd.to_datetime('2017-12-30')
df[0]['transaction_date']  = pd.to_datetime(df[0]['transaction_date'] )
latest_transaction_date = df[0]['transaction_date'].max()
# Calculate the number of days from the customer's last transaction
df_data['Recency'] = specific_date - df[0].groupby('customer_id')['transaction_date'].max()
# Extract the number of days from the timedelta result
df_data['Recency'] = df_data['Recency'].dt.days



df_data.head()

,customer_id,age,gender,job_industry_category,owns_car,past_3_years_bike_related_purchases,state,postcode,wealth_segment,Recency,Profit,Frequency
0,2950,68.0,Male,Financial Services,Yes,19.0,VIC,3064.0,Mass Customer,NaN,17.87,3.0
1,3120,44.0,Female,Health,Yes,89.0,NSW,2196.0,Mass Customer,7.0,1702.55,7.0
2,402,46.0,Male,Retail,No,9.0,NSW,2835.0,Affluent Customer,128.0,1544.61,6.0
3,3135,61.0,Male,Financial Services,No,83.0,NSW,2096.0,Mass Customer,102.0,817.36,7.0
4,787,NaN,NaN,NaN,NaN,NaN,NSW,2292.0,NaN,195.0,1055.82,10.0


In [61]:
print(df[0].isnull().sum())

transaction_id                            0
product_id                                0
customer_id                               0
transaction_date                          0
online_order                              0
order_status                              0
brand                                     0
product_line                              0
product_class                             0
product_size                              0
list_price                                0
standard_cost                             0
product_first_sold_date                   0
gender                                 6418
wealth_segment                         6418
age                                    6418
transaction_date_datetime                 0
job_industry_category                  6418
owns_car                               6418
past_3_years_bike_related_purchases    6418
country                                  28
state                                    28
postcode                        

In [64]:
print(df_data.isnull().sum())
df_data.shape

customer_id                            0
age                                    0
gender                                 0
job_industry_category                  0
owns_car                               0
past_3_years_bike_related_purchases    0
state                                  0
postcode                               0
wealth_segment                         0
Recency                                0
Profit                                 0
Frequency                              0
dtype: int64


(2241, 12)

In [100]:
# df_data = df_data.loc[:, ~df_data.columns.isnull()]
df_data.dropna(inplace=True)
df_data.head()

,customer_id,age,gender,job_industry_category,owns_car,past_3_years_bike_related_purchases,state,postcode,wealth_segment,Recency,Profit,Frequency
1,3120,44.0,Female,Health,Yes,89.0,NSW,2196.0,Mass Customer,7.0,1702.55,7.0
2,402,46.0,Male,Retail,No,9.0,NSW,2835.0,Affluent Customer,128.0,1544.61,6.0
3,3135,61.0,Male,Financial Services,No,83.0,NSW,2096.0,Mass Customer,102.0,817.36,7.0
5,2339,65.0,Male,Property,Yes,3.0,NSW,2153.0,Affluent Customer,16.0,709.34,7.0
6,1542,45.0,Male,Health,Yes,56.0,NSW,2155.0,Mass Customer,64.0,15.08,5.0


Dividing the data into quartiles using the qcut method.
Scores are given from 1-4 ranging from Best to Worst respectively.


- Most Recent Customer will get a score of 1.  
- Most Frequent customer will get a score of 1.
- The customer who made purchases worth more monetary terms gets a score of 1.

- Below are the quartiles for all the three columns. Have a look at the r_quartile, f_quartile and m_quartile to understand how the scores are being assigned with respect to the quartile values.

In [101]:
df_data['Profit'] = pd.to_numeric(df_data['Profit'], errors='coerce')

In [102]:
quartiles = df_data[['Recency','Frequency','Profit']].quantile(q=[0.25,0.50,0.75])
print(quartiles)

      Recency  Frequency  Profit
0.25     18.0        5.0  135.85
0.50     46.0        6.0  445.21
0.75     89.0        8.0  834.94


In [103]:
df_data['r_quartile'] = pd.qcut(df_data['Recency'], 4, ['1','2','3','4'])
df_data['p_quartile'] = pd.qcut(df_data['Profit'], 4, ['4','3','2','1'])
df_data['f_quartile'] = pd.qcut(df_data['Frequency'], 4, ['4','3','2','1'] )

In [104]:
df_data.head()

,customer_id,age,gender,job_industry_category,owns_car,past_3_years_bike_related_purchases,state,postcode,wealth_segment,Recency,Profit,Frequency,r_quartile,p_quartile,f_quartile
1,3120,44.0,Female,Health,Yes,89.0,NSW,2196.0,Mass Customer,7.0,1702.55,7.0,1,1,2
2,402,46.0,Male,Retail,No,9.0,NSW,2835.0,Affluent Customer,128.0,1544.61,6.0,4,1,3
3,3135,61.0,Male,Financial Services,No,83.0,NSW,2096.0,Mass Customer,102.0,817.36,7.0,4,2,2
5,2339,65.0,Male,Property,Yes,3.0,NSW,2153.0,Affluent Customer,16.0,709.34,7.0,1,2,2
6,1542,45.0,Male,Health,Yes,56.0,NSW,2155.0,Mass Customer,64.0,15.08,5.0,3,4,4


Creating a combined score.

In [105]:
df_data['RFM_Score'] = df_data.r_quartile.astype(str)+ df_data.p_quartile.astype(str) + df_data.f_quartile.astype(str)
df_data.head()

,customer_id,age,gender,job_industry_category,owns_car,past_3_years_bike_related_purchases,state,postcode,wealth_segment,Recency,Profit,Frequency,r_quartile,p_quartile,f_quartile,RFM_Score
1,3120,44.0,Female,Health,Yes,89.0,NSW,2196.0,Mass Customer,7.0,1702.55,7.0,1,1,2,112
2,402,46.0,Male,Retail,No,9.0,NSW,2835.0,Affluent Customer,128.0,1544.61,6.0,4,1,3,413
3,3135,61.0,Male,Financial Services,No,83.0,NSW,2096.0,Mass Customer,102.0,817.36,7.0,4,2,2,422
5,2339,65.0,Male,Property,Yes,3.0,NSW,2153.0,Affluent Customer,16.0,709.34,7.0,1,2,2,122
6,1542,45.0,Male,Health,Yes,56.0,NSW,2155.0,Mass Customer,64.0,15.08,5.0,3,4,4,344


The customers are labelled as "Best" and "Others" if their RFM scores are in the following range.
"R_label" will act as the target variable.

In [106]:
def RFMlabel(x):
    if (x['RFM_Score'] in ['111','112','121','211','122','212','221','222','311','312','321','322']): 
        return "Best"
    
    else: 
        return "Others"

In [107]:
data = df_data
data['R_label'] = data.apply(RFMlabel,axis=1)
data

,customer_id,age,gender,job_industry_category,owns_car,past_3_years_bike_related_purchases,state,postcode,wealth_segment,Recency,Profit,Frequency,r_quartile,p_quartile,f_quartile,RFM_Score,R_label
1,3120,44.0,Female,Health,Yes,89.0,NSW,2196.0,Mass Customer,7.0,1702.55,7.0,1,1,2,112,Best
2,402,46.0,Male,Retail,No,9.0,NSW,2835.0,Affluent Customer,128.0,1544.61,6.0,4,1,3,413,Others
3,3135,61.0,Male,Financial Services,No,83.0,NSW,2096.0,Mass Customer,102.0,817.36,7.0,4,2,2,422,Others
5,2339,65.0,Male,Property,Yes,3.0,NSW,2153.0,Affluent Customer,16.0,709.34,7.0,1,2,2,122,Best
6,1542,45.0,Male,Health,Yes,56.0,NSW,2155.0,Mass Customer,64.0,15.08,5.0,3,4,4,344,Others
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3496,1306,67.0,Male,Health,Yes,13.0,QLD,4208.0,Affluent Customer,256.0,1069.55,5.0,4,1,4,414,Others
3497,1913,50.0,Female,Health,Yes,36.0,NSW,2299.0,Mass Customer,52.0,4.80,13.0,3,4,1,341,Others
3498,2197,30.0,Male,Financial Services,No,32.0,VIC,3355.0,Mass Customer,127.0,179.44,3.0,4,3,4,434,Others
3500,1151,58.0,Female,Agriculture,Yes,27.0,NSW,2166.0,High Net Worth,144.0,1660.88,5.0,4,1,4,414,Others


In [72]:
data.to_excel(r'cleaned.xlsx', index=False) 
data.head()

,customer_id,age,gender,job_industry_category,owns_car,past_3_years_bike_related_purchases,state,postcode,wealth_segment,Recency,Profit,Frequency,r_quartile,p_quartile,f_quartile,RFM_Score,R_label
1,3120,44.0,Female,Health,Yes,89.0,NSW,2196.0,Mass Customer,7.0,1702.55,7.0,1,1,2,112,Best
2,402,46.0,Male,Retail,No,9.0,NSW,2835.0,Affluent Customer,128.0,1544.61,6.0,4,1,3,413,Others
3,3135,61.0,Male,Financial Services,No,83.0,NSW,2096.0,Mass Customer,102.0,817.36,7.0,4,2,2,422,Others
5,2339,65.0,Male,Property,Yes,3.0,NSW,2153.0,Affluent Customer,16.0,709.34,7.0,1,2,2,122,Best
6,1542,45.0,Male,Health,Yes,56.0,NSW,2155.0,Mass Customer,64.0,15.08,5.0,3,4,4,344,Others


In [108]:
df2 = pd.read_excel('cleaned.xlsx')

In [109]:
df2.groupby('R_label').count()

,customer_id,age,gender,job_industry_category,owns_car,past_3_years_bike_related_purchases,state,postcode,wealth_segment,Recency,Profit,Frequency,r_quartile,p_quartile,f_quartile,RFM_Score
R_label,,,,,,,,,,,,,,,,
Best,370,370,370,370,370,370,370,370,370,370,370,370,370,370,370,370
Others,1871,1871,1871,1871,1871,1871,1871,1871,1871,1871,1871,1871,1871,1871,1871,1871


Data is divided into segments based on 'Age'. 'Age' is being converted into a categorical value.

In [110]:
df2['age']=pd.cut(df2.age,bins=[15,35,50,90],labels=['Young Adults','Middle Age','Older'])
df2.head()

,customer_id,age,gender,job_industry_category,owns_car,past_3_years_bike_related_purchases,state,postcode,wealth_segment,Recency,Profit,Frequency,r_quartile,p_quartile,f_quartile,RFM_Score,R_label
0,3120,Middle Age,Female,Health,Yes,89,NSW,2196,Mass Customer,7,1702.55,7,1,1,2,112,Best
1,402,Middle Age,Male,Retail,No,9,NSW,2835,Affluent Customer,128,1544.61,6,4,1,3,413,Others
2,3135,Older,Male,Financial Services,No,83,NSW,2096,Mass Customer,102,817.36,7,4,2,2,422,Others
3,2339,Older,Male,Property,Yes,3,NSW,2153,Affluent Customer,16,709.34,7,1,2,2,122,Best
4,1542,Middle Age,Male,Health,Yes,56,NSW,2155,Mass Customer,64,15.08,5,3,4,4,344,Others


### Undersampling

In [111]:
# import library
from imblearn.under_sampling import RandomUnderSampler


X = df2[['age', 'gender', 'state','job_industry_category','past_3_years_bike_related_purchases','wealth_segment','owns_car']].values
y = df2['R_label']
rus = RandomUnderSampler(random_state=42, replacement=True)# fit predictor and target variable
x_rus, y_rus = rus.fit_resample(X, y)
print('original dataset shape:', y.shape)
print('Resample dataset shape', y_rus.shape)

original dataset shape: (2241,)
Resample dataset shape (740,)


The ratio of records with "Best" and "Others" is nearly 1:2.
Therefore to prevent bias, the dominant class "Others" is undersampled.

In [112]:
X=x_rus
y=y_rus

In [88]:
type(X)

numpy.ndarray

### Data Preprocessing

LabelEncoder is used to normalize categorical varaiables.

In [113]:
from sklearn import preprocessing

le_age = preprocessing.LabelEncoder()
le_age.fit(['Young Adults','Middle Age','Older'])
X[:,0] = le_age.transform(X[:,0]) 

le_gen = preprocessing.LabelEncoder()
le_gen.fit(['Female','Male'])
X[:,1] = le_gen.transform(X[:,1]) 


le_state = preprocessing.LabelEncoder()
le_state.fit([ 'NSW', 'VIC', 'QLD'])
X[:,2] = le_state.transform(X[:,2])

le_job = preprocessing.LabelEncoder()
le_job.fit([ 'Agriculture', 'Entertainment', 'Financial Services','Health','IT','Manufacturing','Property','Retail','Telecommunications'])
X[:,3] = le_job.transform(X[:,3])

le_seg = preprocessing.LabelEncoder()
le_seg.fit([ 'Mass Customer','Affluent Customer','High Net Worth'])
X[:,5] = le_seg.transform(X[:,5])

le_car = preprocessing.LabelEncoder()
le_car.fit([ 'Yes','No'])
X[:,6] = le_car.transform(X[:,6])

In [114]:
X[0:20]

array([[0, 0, 0, 3, 89, 2, 1],
       [1, 1, 0, 6, 3, 0, 1],
       [1, 1, 1, 1, 65, 2, 0],
       [1, 1, 0, 7, 73, 2, 0],
       [0, 0, 0, 2, 62, 1, 1],
       [2, 0, 0, 4, 55, 1, 0],
       [0, 1, 1, 2, 13, 2, 1],
       [0, 1, 0, 0, 61, 2, 1],
       [0, 1, 1, 5, 25, 2, 0],
       [0, 0, 0, 0, 71, 1, 0],
       [0, 0, 1, 5, 34, 0, 0],
       [1, 1, 2, 8, 9, 2, 0],
       [2, 0, 2, 7, 74, 1, 1],
       [0, 1, 0, 3, 93, 0, 1],
       [0, 1, 0, 5, 4, 2, 1],
       [0, 1, 1, 1, 99, 0, 1],
       [1, 0, 1, 5, 61, 2, 1],
       [1, 0, 2, 4, 55, 1, 0],
       [1, 0, 0, 5, 67, 1, 0],
       [0, 0, 2, 3, 26, 2, 1]], dtype=object)

## XGBoost Classification

In [115]:
from xgboost import XGBClassifier

In [116]:
xgb = XGBClassifier()

y_t = y.replace('Best',1)
y_t = y_t.replace('Others',0)
y_arr = np.asarray(y_t)

In [117]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
RANDOM_STATE = 55 ## You will pass it to every sklearn call so we ensure reproducibility

X_train, X_test, y_train, y_test = train_test_split(X, y_arr, train_size = 0.8, random_state = RANDOM_STATE)

print(y_arr.shape)
print(X.shape)

(740,)
(740, 7)


In [118]:
xgb.fit(X_train, y_train)

print(f"Metrics train:\n\tAccuracy score: {accuracy_score(xgb.predict(X_train),y_train):.4f}\nMetrics test:\n\tAccuracy score: {accuracy_score(xgb.predict(X_test),y_test):.4f}")

Metrics train:
	Accuracy score: 0.9476
Metrics test:
	Accuracy score: 0.4730


In [85]:
y_predxgb = xgb.predict(X_test) 

In [119]:
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 

In [120]:
print("Confusion Matrix: ", 
        confusion_matrix(y_test, y_predxgb)) 
      
print ("Accuracy : ", 
    accuracy_score(y_test,y_predxgb)*100) 
      
print("Report : ", 
    classification_report(y_test, y_predxgb)) 

Confusion Matrix:  [[32 31]
 [47 38]]
Accuracy :  47.2972972972973
Report :                precision    recall  f1-score   support

           0       0.41      0.51      0.45        63
           1       0.55      0.45      0.49        85

    accuracy                           0.47       148
   macro avg       0.48      0.48      0.47       148
weighted avg       0.49      0.47      0.48       148



## Predicting class labels for New Customers

In [135]:
df[1].head()

,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,address,postcode,state,country,property_valuation,Rank,Value,age
0,Chickie,Brister,Male,86,1957-07-12,General Manager,Manufacturing,Mass Customer,N,Yes,14,45 Shopko Center,4500,QLD,Australia,6,1,1.71875,66
1,Morly,Genery,Male,69,1970-03-22,Structural Engineer,Property,Mass Customer,N,No,16,14 Mccormick Park,2113,NSW,Australia,11,1,1.71875,53
2,Ardelis,Forrester,Female,10,1974-08-28,Senior Cost Accountant,Financial Services,Affluent Customer,N,No,10,5 Colorado Crossing,3505,VIC,Australia,5,1,1.71875,49
3,Lucine,Stutt,Female,64,1979-01-28,Account Representative III,Manufacturing,Affluent Customer,N,Yes,5,207 Annamark Plaza,4814,QLD,Australia,1,4,1.703125,44
4,Melinda,Hadlee,Female,34,1965-09-21,Financial Analyst,Financial Services,Affluent Customer,N,No,19,115 Montana Place,2093,NSW,Australia,9,4,1.703125,58


In [136]:
X_new = df[1][['age', 'gender', 'state','job_industry_category','past_3_years_bike_related_purchases','wealth_segment','owns_car']]
X_new.head()

,age,gender,state,job_industry_category,past_3_years_bike_related_purchases,wealth_segment,owns_car
0,66,Male,QLD,Manufacturing,86,Mass Customer,Yes
1,53,Male,NSW,Property,69,Mass Customer,No
2,49,Female,VIC,Financial Services,10,Affluent Customer,No
3,44,Female,QLD,Manufacturing,64,Affluent Customer,Yes
4,58,Female,NSW,Financial Services,34,Affluent Customer,No


In [137]:
X_new.to_excel(r'newcustomers.xlsx', index=False) 

In [138]:
X_new = pd.read_excel('newcustomers.xlsx')
X_new.head()

,age,gender,state,job_industry_category,past_3_years_bike_related_purchases,wealth_segment,owns_car
0,66,Male,QLD,Manufacturing,86,Mass Customer,Yes
1,53,Male,NSW,Property,69,Mass Customer,No
2,49,Female,VIC,Financial Services,10,Affluent Customer,No
3,44,Female,QLD,Manufacturing,64,Affluent Customer,Yes
4,58,Female,NSW,Financial Services,34,Affluent Customer,No


In [139]:
X_new['age']=pd.cut(X_new.age,bins=[15,35,50,90],labels=['Young Adults','Middle Age','Older'])

In [140]:
Xtest = X_new[['age', 'gender', 'state','job_industry_category','past_3_years_bike_related_purchases','wealth_segment','owns_car']].values
Xtest.shape

(715, 7)

In [141]:
from sklearn import preprocessing

le_age = preprocessing.LabelEncoder()
le_age.fit(['Young Adults','Middle Age','Older'])
Xtest[:,0] = le_age.transform(Xtest[:,0]) 

le_gen = preprocessing.LabelEncoder()
le_gen.fit(['Female','Male'])
Xtest[:,1] = le_gen.transform(Xtest[:,1]) 


le_state = preprocessing.LabelEncoder()
le_state.fit([ 'NSW', 'VIC', 'QLD'])
Xtest[:,2] = le_state.transform(Xtest[:,2])

le_job = preprocessing.LabelEncoder()
le_job.fit([ 'Agriculture', 'Entertainment', 'Financial Services','Health','IT','Manufacturing','Property','Retail','Telecommunications'])
Xtest[:,3] = le_job.transform(Xtest[:,3])

le_seg = preprocessing.LabelEncoder()
le_seg.fit([ 'Mass Customer','Affluent Customer','High Net Worth'])
Xtest[:,5] = le_seg.transform(Xtest[:,5])

le_car = preprocessing.LabelEncoder()
le_car.fit([ 'Yes','No'])
Xtest[:,6] = le_car.transform(Xtest[:,6])

In [142]:
finalypred=xgb.predict(Xtest) 

In [151]:
print(final.shape)
print(finalypred.shape)

(715, 19)
(715,)


In [143]:
final = df[1]
submission = pd.concat([pd.DataFrame(final), pd.DataFrame(finalypred)], axis = 'columns')
submission.head()

,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,address,postcode,state,country,property_valuation,Rank,Value,age,0
0,Chickie,Brister,Male,86,1957-07-12,General Manager,Manufacturing,Mass Customer,N,Yes,14,45 Shopko Center,4500,QLD,Australia,6,1,1.71875,66.0,0.0
1,Morly,Genery,Male,69,1970-03-22,Structural Engineer,Property,Mass Customer,N,No,16,14 Mccormick Park,2113,NSW,Australia,11,1,1.71875,53.0,1.0
2,Ardelis,Forrester,Female,10,1974-08-28,Senior Cost Accountant,Financial Services,Affluent Customer,N,No,10,5 Colorado Crossing,3505,VIC,Australia,5,1,1.71875,49.0,0.0
3,Lucine,Stutt,Female,64,1979-01-28,Account Representative III,Manufacturing,Affluent Customer,N,Yes,5,207 Annamark Plaza,4814,QLD,Australia,1,4,1.703125,44.0,0.0
4,Melinda,Hadlee,Female,34,1965-09-21,Financial Analyst,Financial Services,Affluent Customer,N,No,19,115 Montana Place,2093,NSW,Australia,9,4,1.703125,58.0,0.0


In [144]:
submission.rename(columns = {0:'Prediction'}, inplace = True) 
submission.head()

,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,address,postcode,state,country,property_valuation,Rank,Value,age,Prediction
0,Chickie,Brister,Male,86,1957-07-12,General Manager,Manufacturing,Mass Customer,N,Yes,14,45 Shopko Center,4500,QLD,Australia,6,1,1.71875,66.0,0.0
1,Morly,Genery,Male,69,1970-03-22,Structural Engineer,Property,Mass Customer,N,No,16,14 Mccormick Park,2113,NSW,Australia,11,1,1.71875,53.0,1.0
2,Ardelis,Forrester,Female,10,1974-08-28,Senior Cost Accountant,Financial Services,Affluent Customer,N,No,10,5 Colorado Crossing,3505,VIC,Australia,5,1,1.71875,49.0,0.0
3,Lucine,Stutt,Female,64,1979-01-28,Account Representative III,Manufacturing,Affluent Customer,N,Yes,5,207 Annamark Plaza,4814,QLD,Australia,1,4,1.703125,44.0,0.0
4,Melinda,Hadlee,Female,34,1965-09-21,Financial Analyst,Financial Services,Affluent Customer,N,No,19,115 Montana Place,2093,NSW,Australia,9,4,1.703125,58.0,0.0


In [145]:
submission['Prediction'][submission['Prediction']==1]

1      1.0
11     1.0
14     1.0
17     1.0
18     1.0
      ... 
657    1.0
662    1.0
668    1.0
673    1.0
691    1.0
Name: Prediction, Length: 297, dtype: float64

In [152]:
submission=submission[:][submission['Prediction']==1]
submission.head()

,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,address,postcode,state,country,property_valuation,Rank,Value,age,Prediction
1,Morly,Genery,Male,69,1970-03-22,Structural Engineer,Property,Mass Customer,N,No,16,14 Mccormick Park,2113,NSW,Australia,11,1,1.71875,53.0,1.0
11,Wheeler,Winward,Male,48,1999-08-30,Environmental Specialist,Manufacturing,Mass Customer,N,No,10,3 Golden Leaf Point,3216,VIC,Australia,8,12,1.625,24.0,1.0
14,Mandie,Feares,Female,32,1964-04-19,Clinical Specialist,Health,Mass Customer,N,No,10,39 Kedzie Pass,4053,QLD,Australia,8,14,1.59375,59.0,1.0
17,Winnifred,Beswetherick,Female,83,1976-06-08,Actuary,Financial Services,Mass Customer,N,No,14,61 4th Street,3040,VIC,Australia,10,17,1.546875,47.0,1.0
18,Odilia,Quick,Female,65,1938-11-09,General Manager,Manufacturing,Affluent Customer,N,Yes,11,1550 Russell Way,2222,NSW,Australia,11,19,1.53125,85.0,1.0


In [147]:
submission.to_excel(r'Target Customers.xlsx', index = False)

## TOP 10 valuable customers:

In [156]:
best_customers = pd.read_excel(r'Target Customers.xlsx')
N = 10

for i in range(N):
    print(best_customers.iloc[i]['first_name'], best_customers.iloc[i]['last_name'])

Morly Genery
Wheeler Winward
Mandie Feares
Winnifred Beswetherick
Odilia Quick
Karly Willavize
Teddie Burchill
Dyane Burwell
Claudine Barstowk
Blinnie Roze
